In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import nibabel as nib
import pandas as pd
from torchvision import transforms
from tqdm import tqdm
import os
import datetime
import pandas as pd
import numpy as np
import nibabel as nib
import torch
from torch.utils.data import Dataset
import random

In [2]:
# Set seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [3]:
import importlib.util
import sys

# Specify the full path to the module file
module_path = 'D:\\Github Folder\\MasterThesis\\Code\\FUNCTIONS.py'

# Load the module
spec = importlib.util.spec_from_file_location("FUNCTIONS", module_path)
functions = importlib.util.module_from_spec(spec)
spec.loader.exec_module(functions)

# Now you can use the functions as if you had imported them
load_datasets = functions.load_datasets
create_dataloaders = functions.create_dataloaders
train_and_validate = functions.train_and_validate
test_model = functions.test_model


In [4]:
df = pd.read_excel(r"references\NEW_COMBINED_FINAL_Subject_info.xlsx")
#df = df[df['iteration']=='old']

import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

filtered_df = df.copy()
filtered_df.reset_index(drop=True, inplace=True)

# Initialize the StratifiedShuffleSplit object for the first split (train_val/test)
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Create a new column for the dataset split tags
filtered_df['dataset_split'] = None

# First, split into test and the combined train/validation sets
for train_val_idx, test_idx in splitter.split(filtered_df, filtered_df['Research Group']):
    filtered_df.loc[test_idx, 'dataset_split'] = 'test'
    train_val_set = filtered_df.loc[train_val_idx]

# Now, split the train_val_set into training and validation
splitter = StratifiedShuffleSplit(n_splits=1, test_size=1/8, random_state=42)
for train_idx, val_idx in splitter.split(train_val_set, train_val_set['Research Group']):
    filtered_df.loc[train_val_set.index[train_idx], 'dataset_split'] = 'train'
    filtered_df.loc[train_val_set.index[val_idx], 'dataset_split'] = 'validation'

# Function to balance the classes within each split
def balance_classes(df, column, target):
    groups = df.groupby(column)
    min_size = min(groups.size())
    balanced_df = pd.concat([group.sample(min_size, random_state=42) for name, group in groups])
    return balanced_df

# Apply the balancing function to each dataset split
for split in filtered_df['dataset_split'].unique():
    split_indices = filtered_df[filtered_df['dataset_split'] == split].index
    filtered_df.loc[split_indices] = balance_classes(filtered_df.loc[split_indices], 'Research Group', split)

# Check the distribution of each dataset split within each research group
print(filtered_df['dataset_split'].value_counts())
print(filtered_df.groupby(['Research Group', 'dataset_split']).size())


In [5]:
# Assuming 'df' is your DataFrame loaded with the 'Research Group' column available
label_categories = pd.Categorical(df['Research Group'])
label_mapping = {code: category for code, category in enumerate(label_categories.categories)}

In [6]:
import torch
import torch.nn as nn

class Baseline3DCNN(nn.Module):
    def __init__(self, num_classes=1, init_filters=16, kernel_size=2, stride=2, num_fc_units=128, verbose=False):
        super(Baseline3DCNN, self).__init__()
        self.init_filters = init_filters
        self.kernel_size = kernel_size
        self.stride = stride
        self.num_fc_units = num_fc_units
        self.verbose = verbose

        self.conv1 = nn.Conv3d(1, init_filters, kernel_size=kernel_size, stride=stride, padding=kernel_size//2)
        self.conv2 = nn.Conv3d(init_filters, init_filters*2, kernel_size=kernel_size, stride=stride, padding=kernel_size//2)
        self.conv3 = nn.Conv3d(init_filters*2, init_filters*4, kernel_size=kernel_size, stride=stride, padding=kernel_size//2)
        self.pool = nn.MaxPool3d(2)
        self.relu = nn.ReLU()

        # Initialize example input to determine final dimensions for the FC layer
        example_input = torch.randn(1, 1, 193, 229, 193)
        example_output = self.forward_features(example_input)
        self.final_dim = example_output.numel()

        self.fc1 = nn.Linear(2304, num_fc_units)
        self.fc2 = nn.Linear(num_fc_units, num_classes)

    def forward_features(self, x):
        x = self.conv1(x)
        if self.verbose:
            print("After conv1:", x.shape)
        x = self.relu(x)
        x = self.pool(x)
        if self.verbose:
            print("After pool1:", x.shape)

        x = self.conv2(x)
        if self.verbose:
            print("After conv2:", x.shape)
        x = self.relu(x)
        x = self.pool(x)
        if self.verbose:
            print("After pool2:", x.shape)

        x = self.conv3(x)
        if self.verbose:
            print("After conv3:", x.shape)
        x = self.relu(x)
        x = self.pool(x)
        if self.verbose:
            print("After pool3:", x.shape)

        x = torch.flatten(x, start_dim=1)
        if self.verbose:
            print("After flatten:", x.shape)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = self.fc1(x)
        if self.verbose:
            print("After fc1:", x.shape)
        x = self.relu(x)
        x = self.fc2(x)
        if self.verbose:
            print("Output shape:", x.shape)
        return x




In [7]:
def select_optimizer(model, config):
    """Select optimizer based on configuration."""
    lr = config.get('lr', 0.001)  # Default learning rate
    weight_decay = config.get('weight_decay', 0)  # Default weight decay
    
    if config['optimizer'] == 'Adam':
        return optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif config['optimizer'] == 'SGD':
        momentum = config.get('momentum', 0.9)  # Default momentum for SGD
        return optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    else:
        raise ValueError("Unsupported optimizer")

def select_criterion(config):
    """Select loss criterion based on configuration."""
    if config['loss_criterion'] == 'Cross-Entropy':
        return nn.CrossEntropyLoss()
    elif config['loss_criterion'] == 'BCEWithLogits':
        return nn.BCEWithLogitsLoss()
    else:
        raise ValueError("Unsupported loss criterion")
    
    

In [10]:
from openpyxl import load_workbook
import pandas as pd
import os
import datetime
import torch
import torch.nn as nn
import torch.optim as optim






def run_experiment(df, config):
    train_dataset, val_dataset, test_dataset = load_datasets(df, image_type = config['image_type'], loss_type = config['loss_criterion'])
    train_loader, val_loader, test_loader = create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=config['batch_size'])
    
    device = torch.device("cuda")
    model = Baseline3DCNN().to(device)
    
    criterion = select_criterion(config)
    optimizer = select_optimizer(model, config)


    # Training and validation
    train_accuracies, val_accuracies, val_losses, results_df = train_and_validate(model, train_loader, val_loader, criterion, optimizer, config['num_epochs'], config['patience'], device, config['loss_criterion'])
    test_results, test_accuracy = test_model(model, test_loader, label_mapping, device, loss_type = config['loss_criterion'])
    
    # Save detailed results to Excel
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime('%Y-%m-%d_%H-%M-%S')
    report_filename = os.path.join('reports', f'{formatted_time}_Experiment.xlsx')
    onnx_filename = os.path.join('models', f'{formatted_time}_Model.')
          # Save the model to ONNX


    
    summary_data = {
        'Phase': ['Training', 'Validation', 'Testing'],
        'Accuracy': [train_accuracies[-1], val_accuracies[-1], test_accuracy]
    }
    summary_df = pd.DataFrame(summary_data)
    all_results = pd.DataFrame(test_results)
    config_df = pd.DataFrame([config])
    
    with pd.ExcelWriter(report_filename) as writer:
        config_df.to_excel(writer, sheet_name='Configuration')
        all_results.to_excel(writer, sheet_name='Results')
        summary_df.to_excel(writer, sheet_name='Summary')
        results_df.to_excel(writer, sheet_name='Training_Results')

# Append a summary of this experiment to the cumulative RESULTS.xlsx file
    results_file = os.path.join('reports', 'RESULTS.xlsx')
    experiment_summary = {**config, **{'Training Accuracy': train_accuracies[-1], 'Validation Accuracy': val_accuracies[-1], 'Test Accuracy': test_accuracy, 'DATETIME': formatted_time}}
    summary_row = pd.DataFrame([experiment_summary])

    if os.path.exists(results_file):
        with pd.ExcelWriter(results_file, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            existing_df = pd.read_excel(results_file)
            combined_df = pd.concat([existing_df, summary_row], ignore_index=True)
            combined_df = combined_df.reindex(columns=(existing_df.columns.tolist() + [col for col in summary_row.columns if col not in existing_df.columns]))
            combined_df.to_excel(writer, index=False, sheet_name='Sheet1')
    else:
        summary_row.to_excel(results_file, index=False)

    # Save the model to ONNX
    dummy_input = torch.randn(1, 1, 193, 229, 193, device=device)  # Adjust size according to your model's input
    torch.onnx.export(model, dummy_input, onnx_filename, export_params=True)

    return report_filename, train_accuracies[-1], val_accuracies[-1], test_accuracy


# Example configuration and use case
config = {
    'optimizer': 'Adam',
    'lr' : 0.01,
    'loss_criterion': 'BCEWithLogits',
    'num_epochs': 20,
    'batch_size': 2,
    'Description' : 'TEST',
    'image_type' : 'Fused Images',
    'patience' : 20,
    'model' : 'BASELINE3DCNN',
    'num_filters' : 128,
    'stride' : 2,
    'kernel' : 3
    
    
    
    


}
     #  'Co-registered PET': 'Co-registered PET',
      #      'Fused Images': 'Fused Images',
        #    'Masked PET': 'Masked PET',
       #     'Spatial Normalization': 'Spatial Normalization',
         #   'Resampled Images(Co-registered PET)': 'Resampled Images(Co-registered PET)',
         #   'Resampled Images(Masked PET)': 'Resampled Images(Masked PET)',
        #    'Resampled Images(Spatial Normalization)': 'Resampled Images(Spatial Normalization)',
         #   'Resampled Images_fused': 'Resampled Images_fused'
        #

In [11]:
image_types = ['Co-registered PET', 'Fused Images', 'Masked PET', 'Spatial Normalization']
results = []
for image_type in image_types:
    config['image_type'] = image_type
    print(f"Working on image type: {image_type}")
    result = run_experiment(df, config)
    results.append(result)

Working on image type: Co-registered PET


Epoch 1/20 - Train: 100%|██████████| 157/157 [00:23<00:00,  6.61it/s]


Epoch 1: Train Loss: 52.3811 - Train Accuracy: 48.73%


Epoch 1/20 - Validate: 100%|██████████| 22/22 [00:05<00:00,  4.24it/s]


Epoch 1: Validation Loss: 1.6431 - Validation Accuracy: 47.73%


...

Epoch 20: Train Loss: 0.6983 - Train Accuracy: 49.36%


Epoch 20/20 - Validate: 100%|██████████| 22/22 [00:01<00:00, 19.06it/s]


Epoch 20: Validation Loss: 0.6922 - Validation Accuracy: 52.27%


Testing:  64%|██████▍   | 29/45 [00:05<00:02,  5.78it/s]


TypeError: expected str, bytes or os.PathLike object, not float

# RUN

In [ ]:
config = {
    'optimizer': 'Adam',
    'lr' : 0.001,
    'loss_criterion': 'BCEWithLogits',
    'num_epochs': 20,
    'batch_size': 8,
    'Description' : 'TEST',
    'image_type' : 'Fused Images',
    'patience' : 20,
    'model' : 'BASELINE3DCNN',
    'num_filters' : 128,
    'stride' : 2,
    'kernel' : 3
    
}

image_types = ['Co-registered PET', 'Fused Images', 'Masked PET', 'Spatial Normalization']
results = []
for image_type in image_types:
    config['image_type'] = image_type
    print(f"Working on image type: {image_type}")
    result = run_experiment(df, config)
    results.append(result)

In [158]:
  #column_mapping = {
  #          'Co-registered PET': 'Co-registered PET',
   #         'Fused Images': 'Fused Images',
    #        'Masked PET': 'Masked PET',
     #       'Spatial Normalization': 'Spatial Normalization',
      #      'Resampled Images(Co-registered PET)': 'Resampled Images(Co-registered PET)',
       #     'Resampled Images(Masked PET)': 'Resampled Images(Masked PET)',
        #    'Resampled Images(Spatial Normalization)': 'Resampled Images(Spatial Normalization)',
         #   'Resampled Images_fused': 'Resampled Images_fused'
        #}

image_types = ['Resampled Images_fused','Resampled Images(Co-registered PET)', 'Resampled Images(Masked PET)', 'Resampled Images(Spatial Normalization)']
results = []
for image_type in image_types:
    config['image_type'] = image_type
    print(f"Working on image type: {image_type}")
    result = run_experiment(df, config)
    results.append(result)



Working on image type: Resampled Images_fused


Epoch 1/150 - Train:   0%|          | 0/39 [00:00<?, ?it/s]

Epoch 1/150 - Train: 100%|██████████| 39/39 [00:25<00:00,  1.55it/s]


Epoch 1: Train Loss: 0.7171 - Train Accuracy: 46.79%


Epoch 1/150 - Validate: 100%|██████████| 5/5 [00:01<00:00,  3.58it/s]


...

Epoch 26/150 - Train: 100%|██████████| 39/39 [00:21<00:00,  1.80it/s]


Epoch 26: Train Loss: 0.7038 - Train Accuracy: 46.79%


Epoch 26/150 - Validate: 100%|██████████| 5/5 [00:01<00:00,  2.54it/s]


Epoch 26: Validation Loss: 0.6766 - Validation Accuracy: 55.00%
Early stopping triggered after 26 epochs due to no improvement in validation loss or accuracy.


Testing: 100%|██████████| 11/11 [00:05<00:00,  2.06it/s]
c:\Users\Micha\.pyenv\pyenv-win\versions\3.10.10\lib\site-packages\torch\onnx\symbolic_helper.py:1515: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'dropout' is set to train=True. Exporting with train=True.
  warnings.warn(


In [150]:
import pandas as pd
from tqdm import tqdm
import sys
import os



def run_multiple_experiments(df):
    """Run experiments for all combinations of configurations."""
    optimizers = ['SGD', 'Adam']
    loss_criterions = ['BCEWithLogits']
    learning_rates = [0.001, 0.05, 0.01, 0.1]
    image_types = [
        'Resampled Images_fused','Resampled Images(Co-registered PET)', 'Resampled Images(Masked PET)']

    num_epochs = 200
    patience = 20
    batch_size = 16

    results = []
    total = len(optimizers) * len(loss_criterions) * len(learning_rates) * len(image_types)
    progress_bar = tqdm(total=total, desc='Running Experiments', leave=True)

    for lr in learning_rates:    
        for optimizer in optimizers:
            for loss_criterion in loss_criterions:
                    for image_type in image_types:
                        config = {
                            'optimizer': optimizer,
                            'loss_criterion': loss_criterion,
                            'lr': lr,
                            'num_epochs': num_epochs,
                            'batch_size': batch_size,
                            'patience': patience,
                            'image_type': image_type,
                            'Description': 'Overnight test of learning rates, optimizer and loss criterions'
                        }
                        
                    report_filename, train_acc, val_acc, test_acc = run_experiment(df, config)

                    results.append({
                        'Optimizer': optimizer,
                        'Loss Criterion': loss_criterion,
                        'Learning Rate': lr,
                        'Image Type': image_type,
                        'Report File': report_filename,
                        'Train Accuracy': train_acc,
                        'Validation Accuracy': val_acc,
                        'Test Accuracy': test_acc
                    })
                    progress_bar.update(1)

    progress_bar.close()

    # Convert results to DataFrame and save or return
    results_df = pd.DataFrame(results)
    results_df.to_csv('experiment_results_summary.csv', index=False)
    return results_df


In [151]:
results_df = run_multiple_experiments(df)

Epoch 1/200 - Train: 100%|██████████| 19/19 [00:18<00:00,  1.00it/s]


Epoch 1: Train Loss: 0.7343 - Train Accuracy: 47.04%


Epoch 1/200 - Validate: 100%|██████████| 2/2 [00:01<00:00,  1.77it/s]


Epoch 1: Validation Loss: 0.6671 - Validation Accuracy: 62.50%


Epoch 2/200 - Train: 100%|██████████| 19/19 [00:13<00:00,  1.36it/s]


...

Epoch 22/200 - Train: 100%|██████████| 19/19 [00:11<00:00,  1.59it/s]


Epoch 22: Train Loss: 0.6946 - Train Accuracy: 49.67%


Epoch 22/200 - Validate: 100%|██████████| 2/2 [00:00<00:00,  2.98it/s]


Epoch 22: Validation Loss: 0.6952 - Validation Accuracy: 40.62%
Early stopping triggered after 22 epochs due to no improvement in validation loss or accuracy.


Testing: 100%|██████████| 5/5 [00:01<00:00,  3.63it/s]
c:\Users\Micha\.pyenv\pyenv-win\versions\3.10.10\lib\site-packages\torch\onnx\symbolic_helper.py:1515: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'dropout' is set to train=True. Exporting with train=True.
  warnings.warn(
Running Experiments:  33%|███▎      | 8/24 [46:07<1:32:15, 345.98s/it]
